In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, InputLayer, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [3]:
# Image specifications
img_height = 128
img_width = 128
batch_size = 32

In [4]:
# Prepare ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size = batch_size,
    class_mode = 'binary'
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 194 images belonging to 2 classes.
Found 28 images belonging to 2 classes.


In [6]:
# Define the model architecture
model = Sequential([
    InputLayer(input_shape=(img_height, img_width, 3)),
    
    
    Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [8]:
# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,719,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,680,513 (21.67 MB)

 Trainable params: 5,680,513 (21.67 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
number_of_epochs = 50

In [10]:
# Train the model
history = model.fit(
    train_generator,
    epochs = number_of_epochs,
    validation_data=validation_generator
)

Epoch 1/50


/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 630ms/step - accuracy: 0.4901 - loss: 0.8800 - val_accuracy: 0.4286 - val_loss: 0.7155
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 456ms/step - accuracy: 0.5918 - loss: 0.6877 - val_accuracy: 0.4286 - val_loss: 0.6972
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 481ms/step - accuracy: 0.5343 - loss: 0.6936 - val_accuracy: 0.4286 - val_loss: 0.7074
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 482ms/step - accuracy: 0.6200 - loss: 0.6821 - val_accuracy: 0.4286 - val_loss: 0.7019
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 472ms/step - accuracy: 0.6120 - loss: 0.6861 - val_accuracy: 0.4286 - val_loss: 0.6946
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 453ms/step - accuracy: 0.5529 - loss: 0.6926 - val_accuracy: 0.4286 - val_loss: 0.6948
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 554ms/step - accuracy: 0.6292 - loss: 0.6899 - val_accuracy: 0.4286 - val_loss: 0.6974
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 503ms/step - accuracy: 0.5867 - loss: 0.6883 - val_accuracy: 0.4286 - val_loss: 0.7029
Epo

In [11]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation accuracy: {val_acc}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 0.9075
Validation accuracy: 0.5


In [12]:
# Save the model, optional
model.save(f'Pth_Files/CNN_model_{number_of_epochs}.keras')